Download file from kaggle

In [1]:
# !pip install kaggle
import kaggle 
!kaggle datasets download ankitbansal06/retail-orders/ -f orders.csv 

2024-04-17 12:23:18,502 WARNING Retrying (Retry(total=2, connect=None, read=None, redirect=None, status=None)) after connection broken by 'NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x0000026C69628580>: Failed to resolve 'www.kaggle.com' ([Errno 11001] getaddrinfo failed)")': /api/v1/datasets/metadata/ankitbansal06/retail-orders
2024-04-17 12:23:18,503 WARNING Retrying (Retry(total=1, connect=None, read=None, redirect=None, status=None)) after connection broken by 'NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x0000026C696286A0>: Failed to resolve 'www.kaggle.com' ([Errno 11001] getaddrinfo failed)")': /api/v1/datasets/metadata/ankitbansal06/retail-orders
2024-04-17 12:23:18,503 WARNING Retrying (Retry(total=0, connect=None, read=None, redirect=None, status=None)) after connection broken by 'NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x0000026C69628970>: Failed to resolve 'www.kaggle.com' ([Errno 11001] getaddrinfo f

Extracting zipfile 

In [2]:
import zipfile 

zipfile_ref = zipfile.ZipFile('orders.csv.zip')
zipfile_ref = zipfile_ref.extractall()


Load the dataset to pandas dataframe

In [3]:
import pandas as pd 

df = pd.read_csv('orders.csv')
df.head()

,Order Id,Order Date,Ship Mode,Segment,Country,City,State,Postal Code,Region,Category,Sub Category,Product Id,cost price,List Price,Quantity,Discount Percent
0,1,2023-03-01,Second Class,Consumer,United States,Henderson,Kentucky,42420,South,Furniture,Bookcases,FUR-BO-10001798,240,260,2,2
1,2,2023-08-15,Second Class,Consumer,United States,Henderson,Kentucky,42420,South,Furniture,Chairs,FUR-CH-10000454,600,730,3,3
2,3,2023-01-10,Second Class,Corporate,United States,Los Angeles,California,90036,West,Office Supplies,Labels,OFF-LA-10000240,10,10,2,5
3,4,2022-06-18,Standard Class,Consumer,United States,Fort Lauderdale,Florida,33311,South,Furniture,Tables,FUR-TA-10000577,780,960,5,2
4,5,2022-07-13,Standard Class,Consumer,United States,Fort Lauderdale,Florida,33311,South,Office Supplies,Storage,OFF-ST-10000760,20,20,2,5


Data cleaning

In [4]:
# Cleaning Ship Mode column, replacing 'Not Available', 'unknown' ---> Nan
import numpy as np

df['Ship Mode'].replace({'Not Available':np.nan,'unknown':np.nan},inplace=True)

In [5]:
df['Ship Mode'].unique()

array(['Second Class', 'Standard Class', nan, 'First Class', 'Same Day'],
      dtype=object)

Renaming all columns and converting them in lower case

In [6]:
df.columns = df.columns.str.lower()

In [7]:
df.columns = df.columns.str.replace(' ','_')

In [8]:
df.columns

Index(['order_id', 'order_date', 'ship_mode', 'segment', 'country', 'city',
       'state', 'postal_code', 'region', 'category', 'sub_category',
       'product_id', 'cost_price', 'list_price', 'quantity',
       'discount_percent'],
      dtype='object')

Creating new columns , discountprice ,salesprice,profit

In [9]:
df.head()

,order_id,order_date,ship_mode,segment,country,city,state,postal_code,region,category,sub_category,product_id,cost_price,list_price,quantity,discount_percent
0,1,2023-03-01,Second Class,Consumer,United States,Henderson,Kentucky,42420,South,Furniture,Bookcases,FUR-BO-10001798,240,260,2,2
1,2,2023-08-15,Second Class,Consumer,United States,Henderson,Kentucky,42420,South,Furniture,Chairs,FUR-CH-10000454,600,730,3,3
2,3,2023-01-10,Second Class,Corporate,United States,Los Angeles,California,90036,West,Office Supplies,Labels,OFF-LA-10000240,10,10,2,5
3,4,2022-06-18,Standard Class,Consumer,United States,Fort Lauderdale,Florida,33311,South,Furniture,Tables,FUR-TA-10000577,780,960,5,2
4,5,2022-07-13,Standard Class,Consumer,United States,Fort Lauderdale,Florida,33311,South,Office Supplies,Storage,OFF-ST-10000760,20,20,2,5


In [10]:
# creating discount_price column 
df['discount_price'] = (df['list_price']*df['discount_percent'])/100

In [11]:
# creating salesprice column

df['sales_price'] = df['list_price'] - df['discount_price']

In [12]:
# creating profit column
df['profit'] = df['sales_price'] - df['cost_price']


Dropping unnessesary columns -- discount_percent, cost_price	list_price

In [13]:
df.drop(columns=['cost_price','list_price','discount_percent'],inplace=True)

In [14]:
df.head(2)

,order_id,order_date,ship_mode,segment,country,city,state,postal_code,region,category,sub_category,product_id,quantity,discount_price,sales_price,profit
0,1,2023-03-01,Second Class,Consumer,United States,Henderson,Kentucky,42420,South,Furniture,Bookcases,FUR-BO-10001798,2,5.2,254.8,14.8
1,2,2023-08-15,Second Class,Consumer,United States,Henderson,Kentucky,42420,South,Furniture,Chairs,FUR-CH-10000454,3,21.9,708.1,108.1


Converting order_date from object type to proper datetime format

In [15]:
df.dtypes

order_id            int64
order_date         object
ship_mode          object
segment            object
country            object
city               object
state              object
postal_code         int64
region             object
category           object
sub_category       object
product_id         object
quantity            int64
discount_price    float64
sales_price       float64
profit            float64
dtype: object

In [16]:

df['order_date'] = pd.to_datetime(df['order_date'])

In [17]:
df.dtypes

order_id                   int64
order_date        datetime64[ns]
ship_mode                 object
segment                   object
country                   object
city                      object
state                     object
postal_code                int64
region                    object
category                  object
sub_category              object
product_id                object
quantity                   int64
discount_price           float64
sales_price              float64
profit                   float64
dtype: object

Loading data to sql server

In [18]:
import sqlalchemy as sql

# Use the ODBC DSN instead of specifying the driver directly
# Replace 'YourDSNName' with the name of your ODBC Data Source Name (DSN)
engine = sql.create_engine('mssql://DESKTOP-UD2L1DA\SQLEXPRESS/TestDB?driver=ODBC+DRIVER+17+FOR+SQL+SERVER')



In [19]:
conn = engine.connect()

In [20]:
df.to_sql('df_orders',con=conn,index=False,if_exists= 'append')

38

In [21]:
df.to_csv('df_orders',index=False)